In [ ]:
import sys
import numpy as np
from numpy import loadtxt
import matplotlib.pyplot as plt
from tensorflow.keras.utils import to_categorical
from sklearn.metrics import f1_score
from sklearn.metrics import roc_curve
from sklearn.model_selection import train_test_split
debug = True

f1_scores = []
fpr = []
tpr = []

print('Original Dataset')
amazon = loadtxt("/kaggle/input/train-test-amazon1/train_test_amazon1.sample", delimiter=' ', dtype=np.int64)
ids1 = amazon[:,:2]
meta1 = amazon[:,2:-1]
output1 = amazon[:,-1:]
print("done")


In [ ]:
from sklearn.svm import SVC

from sklearn.preprocessing import LabelEncoder, StandardScaler
import pandas
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import uniform
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_curve, precision_recall_curve, auc

x_train, x_test, y_train, y_test = train_test_split(meta1, output1, test_size = 0.2, random_state=1)
#param_dist = {'C': uniform(0.1, 100)}

#clf = SVC(kernel='rbf', verbose=True)
# Perform random search
#random_search = RandomizedSearchCV(clf, param_distributions=param_dist, n_iter=10, cv=5)
#random_search.fit(x_train, y_train)
print(x_train.shape)
print(x_train)
print(y_train.shape)
print(y_train)
# Best C value
#best_C = random_search.best_params_['C']
#print("best C: ", best_C)
svm = SVC(C=97.95, verbose=True)
svm.fit(x_train, y_train)
y_pred = svm.predict(x_test)
print("prediction:", y_pred)
#y_pred = to_categorical(y_pred)
f1 = f1_score(y_test, y_pred, average='weighted')
# Calculate accuracy and F1 score
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, average='weighted')
recall = recall_score(y_test, y_pred, average='weighted')
fpr, tpr, threshold = roc_curve(y_test, y_pred)

print("Accuracy:", accuracy)
print("F1 Score:", f1)
print("Precision Score:", precision)
print("Recall Score:", recall)
print("FPR Score:", fpr)
print("TPR Score:", tpr)

print (y_pred)
print (y_test)

In [ ]:
from sklearn.metrics import roc_curve, precision_recall_curve, auc

precision, recall, threshold = precision_recall_curve(y_test, y_pred)
prc_auc = auc(recall, precision)
print("Area Under the PR Curve score: ", prc_auc)

plt.figure(figsize=(8,8))
plt.axis([0, 1, 0, 1])  # [xmin, xmax, ymin, ymax]
plt.plot(recall, precision, color='purple', lw=2, label = 'PRC Curve, no S5FCV (area = {:.2f})'.format(prc_auc))
plt.xlabel('Recall')
plt.ylabel('Precision')
plt.title('Precision-Recall Curve, no S5KCV')
plt.legend(loc='upper right')
plt.savefig('/kaggle/working/Amazon_prc_no_S5KCV.png')
plt.show()

In [ ]:
fpr, tpr, threshold = roc_curve(y_test, y_pred)
roc_auc = auc(fpr, tpr)
print("Area Under the ROC Curve score: ", roc_auc)

plt.figure(figsize=(8,8))
plt.plot(fpr, tpr, color='navy', lw=2, label = 'ROC Curve, no S5FCV (area = {:.2f})'.format(roc_auc))
plt.plot([0,1], [0,1], color='lightblue', lw=2, linestyle='--')
plt.axis([0, 1, 0, 1])  # [xmin, xmax, ymin, ymax]
plt.xlabel('FPR')
plt.ylabel('TPR')
plt.title('ROC Curve, no S5KCV')
plt.legend(loc='lower right')
plt.savefig('/kaggle/working/Amazon_roc_no_S5KCV.png')
plt.show()

In [ ]:

# K FOLD 
#from sklearn.model_selection import KFold
#from sklearn.metrics import accuracy_score

# Assuming meta1 contains your features and output1 contains your labels
#X = meta1
#y = output1

# Define number of folds
#k = 5

#kf = KFold(n_splits=k, shuffle=True)

#f1_list = []
#accuracies = []
#precision_l = []
#recall_l = []

#for train_index, test_index in kf.split(X):
#    X_train, X_test = X[train_index], X[test_index]
#    y_train, y_test = y[train_index], y[test_index]
#    
#    svmk = SVC(C=97.95, random_state=42)
#    svmk.fit(X_train, y_train)
#    y_pred = svmk.predict(X_test)
#    
#    f1 = f1_score(y_test, y_pred, average='weighted')
#    accuracy = accuracy_score(y_test, y_pred)
#    precision = precision_score(y_test, y_pred)
#    recall = recall_score(y_test, y_pred)
#    
#    f1_list.append(f1)
#    accuracies.append(accuracy)
#    precision_l.append(precision)
#    recall_l.append(recall)

#avg_f1 = sum(f1_list) / len(f1_list)
#avg_accuracy = sum(accuracies) / len(accuracies)
#avg_precision = sum(precision_l) / len(precision_l)
#avg_recall = sum(recall_l) / len(recall_l)

#print("Average F1 Score:", avg_f1)
#print("Average Accuracy:", avg_accuracy)
#print("Average Precision:", avg_precision)
#print("Average Recall:", avg_recall)


In [ ]:
# STRATIFIED K FOLD 
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_curve

# Assuming meta1 contains your features and output1 contains your labels
X = meta1
y = output1

# Define number of folds
k = 5

kf = StratifiedKFold(n_splits=k, shuffle=True)

f1_list = []
accuracies = []
precision_l = []
recall_l = []
fpr_list = []
tpr_list = []

for train_index, test_index in kf.split(X, y):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    
    svmk2 = SVC(C=97.95, random_state=42)
    svmk2.fit(X_train, y_train)
    y_pred = svmk2.predict(X_test)
 
    f1 = f1_score(y_test, y_pred, average='weighted')
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred, average='weighted')
    recall = recall_score(y_test, y_pred, average='weighted')
    fpr, tpr, threshold = roc_curve(y_test, y_pred)
    
    f1_list.append(f1)
    accuracies.append(accuracy)
    precision_l.append(precision)
    recall_l.append(recall)
    fpr_list.append(fpr)
    tpr_list.append(fpr)

avg_f1 = sum(f1_list) / len(f1_list)
avg_accuracy = sum(accuracies) / len(accuracies)
avg_precision = sum(precision_l) / len(precision_l)
avg_recall = sum(recall_l) / len(recall_l)
#avg_fpr = sum(fpr_list) / len(fpr_list)
#avg_tpr = sum(tpr_list) / len(tpr_list)
#precision = np.concatenate(precision_l)
#recall = np.concatenate(recall_l)

print("Average F1 Score:", avg_f1)
print("Average Accuracy:", avg_accuracy)
print("Average Precision:", avg_precision)
print("Average Recall:", avg_recall)
print("Average FPR:", fpr_list)
print("Average TPR:", tpr_list)

In [ ]:
precision, recall, threshold = precision_recall_curve(y_test, y_pred)
print(recall)
print(precision)
prc_auc = auc(recall, precision)
print("Area Under the PR Curve score: ", prc_auc)

plt.figure(figsize=(8,8))
plt.plot(recall, precision, color='purple', lw=2, label = 'PRC Curve - AK - S5FCV (area = {:.2f})'.format(prc_auc))
plt.axis([0, 1, 0, 1])  # [xmin, xmax, ymin, ymax]
plt.xlabel('Recall')
plt.ylabel('Precision')
plt.title('Precision-Recall Curve, S5KCV')
plt.legend(loc='upper right')
plt.savefig('/kaggle/working/Amazon_prc_S5KCV.png')
plt.show()

In [ ]:
fpr, tpr, threshold = roc_curve(y_test, y_pred)
roc_auc = auc(fpr, tpr)
print("Area Under the ROC Curve score: ", roc_auc)

plt.figure(figsize=(8,8))
plt.plot(fpr, tpr, color='navy', lw=2, label = 'ROC Curve - AK - S5FCV (area = {:.2f})'.format(roc_auc))
plt.axis([0, 1, 0, 1])  # [xmin, xmax, ymin, ymax]
#plt.plot([0,1], [0,1], color='lightblue', lw=2, linestyle='--')
plt.xlabel('FPR')
plt.ylabel('TPR')
plt.title('ROC Curve, no S5KCV')
plt.legend(loc='lower right')
plt.savefig('/kaggle/working/Amazon_roc_S5KCV.png')
plt.show()